In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

<h1>Finding 200 top coins and creating first table</h1>


In [2]:
# Function to parse HTML & return BeautifulSoup object
def HtmlParser(url):
    import requests
    from bs4 import BeautifulSoup
    page = requests.get(url)
    bs = BeautifulSoup(page.content, 'html.parser')
    return bs

In [3]:
#finding the coins on the specified day by url

soup = HtmlParser("https://coinmarketcap.com/historical/20230825/")
soup.body.prettify()
#all the rows of page
AllRows = soup.find_all('tr')
MainLinks = []
Titles = []
for i in range(3 , 203):
  cur = AllRows[i].find('a')
    #first link allways is coin main link
  MainLinks.append(f"https://coinmarketcap.com{cur['href']}")
    #some coins have title tag and for others which don't the title is written on first content
  try:
    Titles.append(cur['title'])
  except:
    Titles.append(cur.contents[0])
SymBols = []
HystLinks = []
for i in range(0 , 200):
  url = MainLinks[i]
  bs = HtmlParser(url)
    #finding symbol based on html pattern
  SymBols.append(bs.find('span' , {'data-role':"coin-symbol"}).contents[0])
  txt = str(bs)
  pos = txt.find("See historical")
    #search for historical data on the entire content of the page
  LinkTxt = txt[pos-200:pos]
  pos2 = LinkTxt.find("href")
    #the link attached to "historical data is written somewhere before
    #200 chars is enough and 100 chars is not for example
  Hlink = LinkTxt[pos2 + 6:]
  Hlink = Hlink[ : Hlink.find(" ") - 1]
    #historical data link starts exactly 6 chars after href and ends after " and an space
  HystLinks.append( f"https://coinmarketcap.com{Hlink}" )

In [4]:
GitHubLinks = []
for i in range(0 , 200):
  url = MainLinks[i]
  bs = HtmlParser(url)
  txt = str(bs)
   #search for github in page
  pos = txt.find("github.com")
  txt2 = txt[pos:]
    #some links end in ) and some in "
  pos2 = txt2.find('"')
  pos3 = txt2.find(')')
  pos2 = min(pos2 , pos3)
  gitlink = txt2[:pos2]
  GitHubLinks.append(gitlink)

In [5]:
df = pd.DataFrame(list(zip(Titles, SymBols , MainLinks , HystLinks , GitHubLinks )),
               columns =['Name', 'Symbol' , 'MainLink' , 'HistoricalLink' , 'GitHubLink'])
df.to_csv("ScrapedData/FirstTable.csv")

In [ ]:
Tags = []
for i in range(0 , 200):
  url = MainLinks[i]
  bs = HtmlParser(url)
    #find tags based on html
  temp = bs.find_all('div' , {'class' : "sc-16891c57-0 sc-b7faf77f-0 jaPKUl"})
  taglist = []
    # no tag
  if len(temp) < 1:
    Tags.append(taglist)
    continue
  for x in temp[0]:
    taglist.append(x.text)
    #the last one is not a tag name and it's common in all coins
  taglist.pop()
  Tags.append(taglist)

In [ ]:
# create List of Tags with CurrencyId from Scraped Data
TagList = []
for i in range(len(Tags)):
    tags = Tags[i]
    for j in range(len(tags)):
        TagList.append((i+1,tags[j]))
# create Dataframe from TagList & save it as csv file
tags = pd.DataFrame(TagList,columns=['CurrencyId','Tag'])
tags.to_csv('ScrapedData/Tags.csv')

In [ ]:
# Get Github Links and replace nan values with empty string
githubs = pd.read_csv('ScrapedData/FirstTable.csv')['GitHubLink']
githubs.fillna('',inplace=True)
# define lists to store scraped data
curr_lang = []
curr_contributors = []
# start scraping
for i in range(len(githubs)):
    if githubs[i] != '':
        # Get Main Github url for Currency
        url_split = githubs[i].split('/',2)
        text = HtmlParser('https://' + url_split[0] + '/' + url_split[1])
        # Scrape Contributors
        Contributors = []
        people_elems = text.findAll('a',class_='member-avatar')
        for element in people_elems:
            Contributors.append(element['href'])
        # Add Contributors's Github to list with CurrencyId
        for contributor in Contributors:
            user_git = 'https://github.com' + contributor
            curr_contributors.append((i+1,user_git))
        # Scrape Languages
        languages = []
        top_lang = text.find('h4',string='Top languages')
        if top_lang != None:
            top_lang_elems = top_lang.parent()
        for elem in top_lang_elems:
            lang_elem = elem.find('span',{'itemprop':"programmingLanguage"})
            if lang_elem != None:
                lang = lang_elem.text
                if lang not in languages:
                    languages.append(lang)
        # Add Languages to list with CurrencyId
        for lang in languages:
           curr_lang.append((i+1,lang)) 

# Convert Lists To Dataframe
CurrencyLanguages = pd.DataFrame(curr_lang,columns=['CurrencyId','Language'])
CurrencyContributors = pd.DataFrame(curr_contributors,columns=['CurrencyId','Contributor'])
# Save Dataframes as csv
CurrencyLanguages.to_csv('ScrapedData/CurrencyLanguages.csv')
CurrencyContributors.to_csv('ScrapedData/CurrencyContributors.csv')

<h1>finding last year data for each coin based on crawled historical links</h1>

In [ ]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
chrome_options = webdriver.ChromeOptions()
base_url = os.getcwd()
prefs = {'download.default_directory' : base_url + '\\HistoricalData'}
chrome_options.add_experimental_option('prefs', prefs)
driver = webdriver.Chrome(chrome_options)
driver.maximize_window()
wait = WebDriverWait(driver, 10)

In [ ]:
#reading data from pre created csv from previous part dataframe to don't run the previous part every time
df = pd.read_csv("ScrapedData/FirstTable.csv")
historical_links = df['HistoricalLink']

In [ ]:
for url in historical_links:
    driver.get(url)
    #click on calender
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]'))).click()
    #go to selecting year page
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    #select start year
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/span[6]'))).click()
    #select start month
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/span[8]'))).click()
    #select start day
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[2]/div[4]/div[5]'))).click()
    #go to selecting year page
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    #select end year
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/span[8]'))).click()
    #select end month
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/span[8]'))).click()
    #select end day
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[2]/div[4]/div[6]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[2]/span/button'))).click()
    #waiting for the site to load csv and download
    wait.until(EC.visibility_of_element_located((By.XPATH, '//tbody/tr[2]')))
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]'))).click()
